# Backtesting Notebook
by **@vaidab**
Version 1.4

### IDEAS
* storing values outside of notebook to be accessed by a plotter notebook https://stackoverflow.com/questions/21248345/is-there-a-way-to-run-multiple-cells-simultaneously-in-ipython-notebook

### Things to Note

* This setup uses the strategy's .json files as it uses them to modify values for testing so it will remove and leave .json files in strategies directory. Remove them with update_params()
* I'm assuming the strategy is good (you've checked the Sortino, Expectancy, etc etc and you're just optimizing the parameters).
* I'm hyperopting outside of the notebook and pasting the results in a raw field because sometimes hyperopting doesn't work inside.

### Process

- [ ] Optional: exit after 5 candles to test the basics, then optimize for SL
- [ ] Go through the smaller TFs (UP, DOWN, SIDE) until the strategy works ok
- [ ] Test it on BTC_ONLY, then on FULL
- [ ] Bruteforce timeframes to get the top one
- [ ] Load strategy in FreqUI and improve it
- [ ] Retest everything
- [ ] Test on R21TD and YTD
- [ ] Hyperopt (go through ProfitSquareSortinoSpringEfficiencyHyperOptLoss, SortinoHyperOptLossDaily, SortinoHyperOptLoss)
- [ ] Retest on optimal TF on ALL, R21TD, YTD
- [ ] Retest with pre-trade checks (like age_filter)
- [ ] Hyperopt protections, trades
- [ ] Retest + 1m timeframe
- [ ] Test on R21TD and YTD for blacklisting pairs
- [ ] Add shorting + redo hyperopt & retests

## Results

### Mass testing to get the best timeframe

| Strategy | Params | Timeframe | Result |
|----------|----------|----------|----------|
| BTC: L: breakout E: 5 candles | 20 | 1d | Negative |
| ALL: L: breakout E: 5 candles | 20 | 1d - * | Positive |
| ALL: L: breakout E: breakout_end | 20 | * | Positive |
| ALL: L: breakout E: support breakout | 20 | * | Positive |

Top timeframe(s): 12h and 1d and 8h for exit @ support breakout

### Default vs hyperopted parameters

| Timeframe | Timerange | Params | Result |
|----------|----------|----------|----------|
| 1d | FULL | 20 | 543% |
| 1d | FULL | 17 | 595% |
| 1d | R21TD | 20 | 1058% |
| 1d | R21TD | 17 | 1723% |
| 1d | YTD | 20 | 692% |
| 1d | YTD | 17 | 380% |
| 12h | FULL | 20 | 695% |
| 12h | FULL | 23 | 756% |
| 12h | R21TD | 20 | 3874% |
| 12h | R21TD | 23 | 2134% |
| 12h | YTD | 20 | 379% |
| 12h | YTD | 23 | 451% |
| 8h | FULL | 20 | -114% |
| 8h | FULL | 27 | 1121% |
| 8h | R21TD | 20 | 4379% |
| 8h | R21TD | 27 | 3861% |
| 8h | YTD | 20 | 390% |
| 8h | YTD | 27 | 439% |

Note that this section is not conclusive for the optimal parameters for the strat. R21TD is a long timerange which probably catches a lot of volatility / market spikes that could've gone either way. It's just to confirm the direction is good.

### Default vs hyperopted vs protections

| Timeframe | Protections |
|----------|----------|
| 1d | yes |
| 12h | no |
| 8h | maybe no |

## TODO
- [X] Optional: exit after 5 candles to test the basics, then optimize for SL
- [X] Go through the smaller TFs (UP, DOWN, SIDE) until the strategy works ok
- [X] Test it on BTC_ONLY, then on FULL
- [X] Bruteforce timeframes to get the top one
- [X] Load strategy in FreqUI and improve it
- [X] Retest everything
- [X] Test on R21TD and YTD
- [X] Hyperopt (go through ProfitSquareSortinoSpringEfficiencyHyperOptLoss, SortinoHyperOptLossDaily, SortinoHyperOptLoss)
- [X] Retest on optimal TF on ALL, R21TD, YTD
- [X] Retest with pre-trade checks (like age_filter)
- [X] Hyperopt protections, trades
- [X] Retest + 1m timeframe
- [X] Test on R21TD and YTD for blacklisting pairs
- [ ] Add shorting + redo hyperopt & retests

# Configuration

In [2]:
MAINCONF="user_data/config/config-backtesting.json"

UP="20230105-20230415"
SIDE="20220108-20220420"
DOWN="20220322-20220622"
FULL="20210808-20220106"
YTD="20230101-"
R21TD="20210101-"

BTC_ONLY="--pairs BTC/USDT:USDT"

In [3]:
# MAIN CONFIGURATION
STRATEGY="Erumperator"
STRATEGY_JSON=f"user_data/strategies/{STRATEGY}.json"
TIMEFRAME_DETAIL="" # "--timeframe-detail 1m"

# Functions

## Default functions

In [96]:
import concurrent.futures
import subprocess
import json
import os, sys
import re
import time
from colorama import Fore, Back, Style

def timer_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()

        elapsed_time_seconds = end_time - start_time
        if elapsed_time_seconds < 60:
            print(f"Function took {Fore.RED}{elapsed_time_seconds:.2f}{Style.RESET_ALL} seconds to execute.")
        elif elapsed_time_seconds < 3600:
            elapsed_time_minutes = elapsed_time_seconds / 60.0
            print(f"Function took {Fore.RED}{elapsed_time_minutes:.2f}{Style.RESET_ALL} minutes to execute.")
        else:
            elapsed_time_hours = elapsed_time_seconds / 3600.0
            print(f"Function took {Fore.RED}{elapsed_time_hours:.2f}{Style.RESET_ALL} hours to execute.")
        return result
    return wrapper

@timer_decorator
def run(command):
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0: raise(RuntimeError('Command failed: ' + result.stderr))
    return result.stdout

@timer_decorator
def run_live(command):
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    printed_lines = set()
    while True:
        output = process.stdout.readline().decode()
        if output == '' and process.poll() is not None:
            break
        if output and not any(word in output for word in ["INFO", "mark", "funding", "FUTURES", "optimizer/optimizer.py:44"]) and output not in printed_lines:
            print(output.strip())
            printed_lines.add(output)
    rc = process.poll()
    return rc

def bruteforce(timeframes):
    def run_tf(timeframe):
        cmd = f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timerange {TIMERANGE} --breakdown month --timeframe {timeframe}"
        return run(cmd)

    with concurrent.futures.ThreadPoolExecutor() as executor:
        # futures = {executor.submit(run, cmd): name for name, cmd in commands_dict.items()}
        futures = {executor.submit(run_tf, tf): tf for tf in timeframes}
        for future in concurrent.futures.as_completed(futures):
            print(f"Command for timeframe {futures[future]} finished.")
            print(extract_summary(future.result()))

def extract_summary(output):   
    lines = output.strip().split('\n')
    relevant_lines = lines[-8:-2]
    parsed_output = '\n'.join(relevant_lines)
    return parsed_output
    
def extract_monthly(output):
    start_marker = "======================= MONTH BREAKDOWN ========================"
    append_text = "\n================================================================"
    end_marker_pattern = r"\n=+ SUMMARY METRICS =+"

    start_pos = output.find(start_marker)
    
    end_marker_match = re.search(end_marker_pattern, output)
    if end_marker_match is None:
        raise ValueError(f"Could not find {end_marker_pattern} in output.")
    end_pos = end_marker_match.start()

    section = output[start_pos:end_pos] + append_text

    colored_section = section.replace("MONTH BREAKDOWN", f"{Fore.GREEN}MONTH BREAKDOWN{Style.RESET_ALL}")
    colored_section = colored_section.replace("STRATEGY SUMMARY", f"{Fore.GREEN}SUMMARY METRICS{Style.RESET_ALL}")
    return colored_section

def extract_best_result(output):
    lines = output.splitlines()
    start_index = next((i for i, line in enumerate(lines) if line.startswith("Best result:")), None)

    if start_index is None:
        raise ValueError("No 'Best result:' line found in output")

    best_result_lines = lines[start_index:]
    best_result_lines_str = '\n'.join(best_result_lines)
    return best_result_lines_str

def extract_worst_pairs(output):
    BOTTOM_X_LINES=5
    CURRENCY="/USDT"
    
    start_marker_pattern = r"\n=+ BACKTESTING REPORT =+"
    end_marker_pattern = r"\n=+ LEFT OPEN TRADES REPORT =+"

    start_marker_match = re.search(start_marker_pattern, output)
    if start_marker_match is None:
        raise ValueError(f"Could not find {start_marker_pattern} in output.")
    start_pos = start_marker_match.start()
    
    end_marker_match = re.search(end_marker_pattern, output)
    if end_marker_match is None:
        raise ValueError(f"Could not find {end_marker_pattern} in output.")
    end_pos = end_marker_match.start()

    append_text = "=" * (len(start_marker_match.group())-1)
    section = output[start_pos:end_pos]
    
    section_lines = section.split('\n')
    usdt_lines = [line for line in section_lines if CURRENCY in line][-BOTTOM_X_LINES:]
    non_usdt_lines = [line for line in section_lines if (CURRENCY not in line) and ("TOTAL" not in line)]

    result_lines = non_usdt_lines + usdt_lines + [append_text]
    result = '\n'.join(result_lines)

    colored_section = result.replace("MONTH BREAKDOWN", f"{Fore.GREEN}MONTH BREAKDOWN{Style.RESET_ALL}")
    colored_section = colored_section.replace("STRATEGY SUMMARY", f"{Fore.GREEN}SUMMARY METRICS{Style.RESET_ALL}")
        
    return colored_section

## User editable functions

In [26]:
def update_params(BL=None):
    if os.path.exists(STRATEGY_JSON):
        os.remove(STRATEGY_JSON)

    if BL is None:
        return(f"Removed {Fore.RED}{STRATEGY_JSON}{Style.RESET_ALL}")
        return

    params = {
      "strategy_name": STRATEGY,
      "params": {
        "buy": {
          "breakout_length": BL
        }
      }
    }

    with open(STRATEGY_JSON, 'w') as json_file:
        json.dump(params, json_file)

    return(f"Params: {Fore.YELLOW}BL {BL}{Style.RESET_ALL}")


os.chdir("/freqtrade")

# Mass testing to get the best timeframe

In [50]:
# FULL: L: breakout E: breakout finished
# P: 20

TIMERANGE=FULL
TIMEFRAMES=["1d", "12h", "8h", "4h"]

update_params()
bruteforce(TIMEFRAMES)

Function took 11.87 seconds to execute.
Command for timeframe 8h finished.
Backtested 2021-08-08 00:00:00 -> 2022-01-06 00:00:00 | Max open trades : 37
========================================================================== STRATEGY SUMMARY =========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |    Avg Duration |   Win  Draw  Loss  Win% |             Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+-----------------+-------------------------+----------------------|
| Erumperator |       212 |           1.12 |         237.50 |           216.990 |          21.70 | 4 days, 7:35:00 |    74     0   138  34.9 | 230.719 USDT  21.73% |
Function took 12.58 seconds to execute.
Command for timeframe 1d finished.
Backtested 2021-08-08 00:00:00 -> 2022-01-06 00:00:00 | Max open trades : 37
================================

In [52]:
# FULL: L: breakout E: support breakout
# P: 20

TIMERANGE=FULL
TIMEFRAMES=["1d", "12h", "8h", "4h"]

update_params()
bruteforce(TIMEFRAMES)

Function took 17.53 seconds to execute.
Command for timeframe 1d finished.
Backtested 2021-08-08 00:00:00 -> 2022-01-06 00:00:00 | Max open trades : 37
=========================================================================== STRATEGY SUMMARY ==========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |      Avg Duration |   Win  Draw  Loss  Win% |             Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+-------------------+-------------------------+----------------------|
| Erumperator |        37 |           7.68 |         284.12 |           261.137 |          26.11 | 22 days, 12:58:00 |    18     0    19  48.6 | 192.856 USDT  19.89% |
Function took 19.38 seconds to execute.
Command for timeframe 12h finished.
Backtested 2021-08-08 00:00:00 -> 2022-01-06 00:00:00 | Max open trades : 37
=======================

# Initial backtests without protections

## 1d

In [51]:
TIMEFRAME="1d"
TIMERANGE=FULL

print(f"Testing: {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@FULL without filters and protections{Style.RESET_ALL}")
print(update_params(20))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing: Erumperator on 1d@FULL without filters and protections
Params: BL 20 LATR 44 LMULT 1.7
Strategy: buy on resistance_breakout and sell on breakout end
Function took 23.60 seconds to execute.
Backtested 2021-08-08 00:00:00 -> 2022-01-06 00:00:00 | Max open trades : 37
=========================================================================== STRATEGY SUMMARY ==========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |      Avg Duration |   Win  Draw  Loss  Win% |             Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+-------------------+-------------------------+----------------------|
| Erumperator |        57 |           9.53 |         543.15 |           511.848 |          51.18 | 14 days, 14:19:00 |    23     0    34  40.4 | 168.013 USDT  16.49% |


In [56]:
TIMEFRAME="1d"
TIMERANGE=R21TD

print(f"Testing: {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@R21TD without filters and protections{Style.RESET_ALL}")
print(update_params(20))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing: Erumperator on 1d@R21TD without filters and protections
Params: BL 20 LATR 44 LMULT 1.7
Strategy: buy on resistance_breakout and sell on breakout end
Function took 26.92 seconds to execute.
Backtested 2021-02-14 00:00:00 -> 2023-07-27 00:00:00 | Max open trades : 48
=========================================================================== STRATEGY SUMMARY ==========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |      Avg Duration |   Win  Draw  Loss  Win% |             Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+-------------------+-------------------------+----------------------|
| Erumperator |       529 |           2.00 |        1058.28 |          1021.823 |         102.18 | 10 days, 15:20:00 |   138     0   391  26.1 | 949.727 USDT  45.16% |


In [55]:
TIMEFRAME="1d"
TIMERANGE=YTD

print(f"Testing: {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@YTD without filters and protections{Style.RESET_ALL}")
print(update_params(20))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing: Erumperator on 1d@YTD without filters and protections
Params: BL 20 LATR 44 LMULT 1.7
Strategy: buy on resistance_breakout and sell on breakout end
Function took 17.78 seconds to execute.
Backtested 2023-01-01 00:00:00 -> 2023-07-27 00:00:00 | Max open trades : 48
=========================================================================== STRATEGY SUMMARY ==========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |      Avg Duration |   Win  Draw  Loss  Win% |             Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+-------------------+-------------------------+----------------------|
| Erumperator |       129 |           5.37 |         692.75 |           681.243 |          68.12 | 12 days, 15:38:00 |    45     0    84  34.9 | 208.199 USDT  13.79% |


## 12h

In [58]:
TIMEFRAME="12h"
TIMERANGE=FULL

print(f"Testing: {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@FULL without filters and protections{Style.RESET_ALL}")
print(update_params(20))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing: Erumperator on 12h@FULL without filters and protections
Params: BL 20 LATR 44 LMULT 1.7
Strategy: buy on resistance_breakout and sell on breakout end
Function took 15.55 seconds to execute.
Backtested 2021-08-08 00:00:00 -> 2022-01-06 00:00:00 | Max open trades : 37
========================================================================== STRATEGY SUMMARY ==========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |     Avg Duration |   Win  Draw  Loss  Win% |             Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+------------------+-------------------------+----------------------|
| Erumperator |       156 |           4.46 |         695.07 |           674.248 |          67.42 | 4 days, 19:09:00 |    37     0   119  23.7 | 193.777 USDT  19.43% |


In [60]:
TIMEFRAME="12h"
TIMERANGE=R21TD

print(f"Testing: {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@R21TD without filters and protections{Style.RESET_ALL}")
print(update_params(20))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing: Erumperator on 12h@R21TD without filters and protections
Params: BL 20 LATR 44 LMULT 1.7
Strategy: buy on resistance_breakout and sell on breakout end
Function took 58.36 seconds to execute.
Backtested 2021-01-23 00:00:00 -> 2023-07-27 12:00:00 | Max open trades : 48
=========================================================================== STRATEGY SUMMARY ==========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |     Avg Duration |   Win  Draw  Loss  Win% |              Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+------------------+-------------------------+-----------------------|
| Erumperator |      1917 |           2.03 |        3896.78 |          3874.671 |         387.47 | 5 days, 18:20:00 |   580     0  1337  30.3 | 1335.809 USDT  25.74% |


In [45]:
TIMEFRAME="12h"
TIMERANGE=YTD

print(f"Testing: {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@R21TD without filters and protections{Style.RESET_ALL}")
print(update_params(20))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing: Erumperator on 12h@R21TD without filters and protections
Params: BL 20
Strategy: buy on resistance_breakout and sell on breakout end
Function took 33.91 seconds to execute.
Backtested 2023-01-01 00:00:00 -> 2023-07-27 12:00:00 | Max open trades : 48
========================================================================== STRATEGY SUMMARY =========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |    Avg Duration |   Win  Draw  Loss  Win% |             Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+-----------------+-------------------------+----------------------|
| Erumperator |       240 |           1.62 |         388.27 |           379.108 |          37.91 | 6 days, 7:33:00 |    79     0   161  32.9 | 259.373 USDT  18.01% |


## 8h

In [14]:
TIMEFRAME="8h"
TIMERANGE=FULL

print(f"Testing: {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@FULL without filters and protections{Style.RESET_ALL}")
print(update_params(20))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on support_breakout{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing: Erumperator on 1h1h@ALL_FULL without filters
Params: BL 20 LATR 44 LMULT 1.7
Strategy: buy on resistance_breakout + upward spike and sell on breakout end
Function took 25.69 seconds to execute.
Backtested 2021-08-08 00:00:00 -> 2022-01-06 00:00:00 | Max open trades : 31
========================================================================= STRATEGY SUMMARY ========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |   Avg Duration |   Win  Draw  Loss  Win% |            Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+----------------+-------------------------+---------------------|
| Erumperator |       173 |          -0.66 |        -114.51 |          -134.933 |         -13.49 |       22:03:00 |    38     0   135  22.0 | 231.35 USDT  21.40% |


In [62]:
TIMEFRAME="8h"
TIMERANGE=R21TD

print(f"Testing: {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@R21TD without filters and protections{Style.RESET_ALL}")
print(update_params(20))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on support_breakout{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing: Erumperator on 8h@R21TD without filters and protections
Params: BL 20 LATR 44 LMULT 1.7
Strategy: buy on resistance_breakout and sell on support_breakout
Function took 1.41 minutes to execute.
Backtested 2021-01-15 16:00:00 -> 2023-07-27 16:00:00 | Max open trades : 48
=========================================================================== STRATEGY SUMMARY ==========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |     Avg Duration |   Win  Draw  Loss  Win% |              Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+------------------+-------------------------+-----------------------|
| Erumperator |      1665 |           2.63 |        4379.20 |          4314.885 |         431.49 | 7 days, 23:40:00 |   440     0  1225  26.4 | 1155.345 USDT  25.70% |


In [61]:
TIMEFRAME="8h"
TIMERANGE=YTD

print(f"Testing: {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@YTD without filters and protections{Style.RESET_ALL}")
print(update_params(20))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on support_breakout{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing: Erumperator on 8h@YTD without filters and protections
Params: BL 20 LATR 44 LMULT 1.7
Strategy: buy on resistance_breakout and sell on support_breakout
Function took 23.63 seconds to execute.
Backtested 2023-01-01 00:00:00 -> 2023-07-27 16:00:00 | Max open trades : 48
========================================================================== STRATEGY SUMMARY ==========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |     Avg Duration |   Win  Draw  Loss  Win% |             Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+------------------+-------------------------+----------------------|
| Erumperator |       223 |           1.75 |         390.00 |           383.699 |          38.37 | 7 days, 13:38:00 |    55     0   168  24.7 | 354.363 USDT  23.63% |


# Hyperopt

In [ ]:
TIMEFRAME="1d"
TIMERANGE=FULL
SPACES="buy"
HYPEROPT_LOSS="ProfitSquareSortinoSpringEfficiencyHyperOptLoss"

print(f"Optimizing {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMERANGE}{Style.RESET_ALL}@FULL without filters or protections{Style.RESET_ALL}")
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade hyperopt --dmmp -j -2 -e 200 --disable-param-export --hyperopt-loss {HYPEROPT_LOSS} "\
    f"--strategy {STRATEGY} --config {MAINCONF} --spaces {SPACES} --timeframe {TIMEFRAME} --timerange {TIMERANGE}"

print(cmd)
output = run(cmd)
print(extract_best_result(output))

Optimizing Erumperator on 20210808-20220106@FULL without filters or protections
Strategy: buy on resistance_breakout and sell on breakout end
freqtrade hyperopt --dmmp -j -2 -e 500 --disable-param-export --hyperopt-loss ProfitSquareSortinoSpringEfficiencyHyperOptLoss --strategy Erumperator --config user_data/config/config-backtesting.json --spaces buy --timeframe 1d --timerange 20210808-20220106


In [5]:
TIMEFRAME="12h"
TIMERANGE=FULL
SPACES="buy"
HYPEROPT_LOSS="ProfitSquareSortinoSpringEfficiencyHyperOptLoss"

print(f"Optimizing {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMERANGE}{Style.RESET_ALL}@FULL without filters or protections{Style.RESET_ALL}")
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade hyperopt --dmmp -j -2 -e 200 --disable-param-export --hyperopt-loss {HYPEROPT_LOSS} "\
    f"--strategy {STRATEGY} --config {MAINCONF} --spaces {SPACES} --timeframe {TIMEFRAME} --timerange {TIMERANGE}"

print(cmd)
# output = run_live(cmd)
# print(extract_best_result(output))

Optimizing Erumperator on 20210808-20220106@FULL without filters or protections
Strategy: buy on resistance_breakout and sell on breakout end
freqtrade hyperopt --dmmp -j -2 -e 500 --disable-param-export --hyperopt-loss ProfitSquareSortinoSpringEfficiencyHyperOptLoss --strategy Erumperator --config user_data/config/config-backtesting.json --spaces buy --timeframe 12h --timerange 20210808-20220106


In [5]:
TIMEFRAME="8h"
TIMERANGE=FULL
SPACES="buy"
HYPEROPT_LOSS="ProfitSquareSortinoSpringEfficiencyHyperOptLoss"

print(f"Optimizing {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMERANGE}{Style.RESET_ALL}@FULL without filters or protections{Style.RESET_ALL}")
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on support breakout start{Style.RESET_ALL}")

cmd=f"freqtrade hyperopt --dmmp -j -2 -e 200 --disable-param-export --hyperopt-loss {HYPEROPT_LOSS} "\
    f"--strategy {STRATEGY} --config {MAINCONF} --spaces {SPACES} --timeframe {TIMEFRAME} --timerange {TIMERANGE}"

print(cmd)
# output = run_live(cmd)
# print(extract_best_result(output))

Optimizing Erumperator on 20210808-20220106@FULL without filters or protections
Strategy: buy on resistance_breakout and sell on support breakout start
freqtrade hyperopt --dmmp -j -2 -e 200 --disable-param-export --hyperopt-loss ProfitSquareSortinoSpringEfficiencyHyperOptLoss --strategy Erumperator --config user_data/config/config-backtesting.json --spaces buy --timeframe 8h --timerange 20210808-20220106


# Retest on optimal TF after HYPEROPT

## 1d

In [37]:
TIMEFRAME="1d"
TIMERANGE=FULL

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@FULL without filters and protections{Style.RESET_ALL}")
print(update_params(17))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing hyperopted Erumperator on 1d@FULL without filters and protections
Params: BL 17
Strategy: buy on resistance_breakout and sell on breakout end
Function took 14.67 seconds to execute.
Backtested 2021-08-08 00:00:00 -> 2022-01-06 00:00:00 | Max open trades : 37
========================================================================== STRATEGY SUMMARY ==========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |     Avg Duration |   Win  Draw  Loss  Win% |             Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+------------------+-------------------------+----------------------|
| Erumperator |        94 |           6.33 |         595.12 |           570.607 |          57.06 | 9 days, 20:56:00 |    36     0    58  38.3 | 121.321 USDT  11.08% |


In [39]:
TIMEFRAME="1d"
TIMERANGE=R21TD

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@R21TD without filters and protections{Style.RESET_ALL}")
print(update_params(17))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing hyperopted Erumperator on 1d@R21TD without filters and protections
Params: BL 17
Strategy: buy on resistance_breakout and sell on breakout end
Function took 27.14 seconds to execute.
Backtested 2021-02-14 00:00:00 -> 2023-07-27 00:00:00 | Max open trades : 48
========================================================================== STRATEGY SUMMARY ==========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |     Avg Duration |   Win  Draw  Loss  Win% |             Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+------------------+-------------------------+----------------------|
| Erumperator |       684 |           2.52 |        1723.81 |          1701.763 |         170.18 | 10 days, 6:13:00 |   211     0   473  30.8 | 893.173 USDT  31.19% |


In [40]:
TIMEFRAME="1d"
TIMERANGE=YTD

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@YTD without filters and protections{Style.RESET_ALL}")
print(update_params(17))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing hyperopted Erumperator on 1d@YTD without filters and protections
Params: BL 17
Strategy: buy on resistance_breakout and sell on breakout end
Function took 16.70 seconds to execute.
Backtested 2023-01-01 00:00:00 -> 2023-07-27 00:00:00 | Max open trades : 48
=========================================================================== STRATEGY SUMMARY ==========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |      Avg Duration |   Win  Draw  Loss  Win% |             Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+-------------------+-------------------------+----------------------|
| Erumperator |       138 |           2.75 |         380.15 |           372.082 |          37.21 | 10 days, 13:13:00 |    47     0    91  34.1 | 223.477 USDT  17.32% |


## 12h

In [36]:
TIMEFRAME="12h"
TIMERANGE=FULL

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@FULL without filters and protections{Style.RESET_ALL}")
print(update_params(23))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing hyperopted Erumperator on 12h@FULL without filters and protections
Params: BL 23
Strategy: buy on resistance_breakout and sell on breakout end
Function took 17.24 seconds to execute.
Backtested 2021-08-08 00:00:00 -> 2022-01-06 00:00:00 | Max open trades : 37
========================================================================== STRATEGY SUMMARY =========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |     Avg Duration |   Win  Draw  Loss  Win% |            Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+------------------+-------------------------+---------------------|
| Erumperator |       166 |           4.56 |         756.90 |           693.768 |          69.38 | 5 days, 22:29:00 |    49     0   117  29.5 | 145.65 USDT  14.07% |


In [41]:
TIMEFRAME="12h"
TIMERANGE=R21TD

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@R21TD without filters and protections{Style.RESET_ALL}")
print(update_params(23))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing hyperopted Erumperator on 12h@R21TD without filters and protections
Params: BL 23
Strategy: buy on resistance_breakout and sell on breakout end
Function took 50.64 seconds to execute.
Backtested 2021-01-23 00:00:00 -> 2023-07-27 12:00:00 | Max open trades : 48
========================================================================== STRATEGY SUMMARY ==========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |    Avg Duration |   Win  Draw  Loss  Win% |              Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+-----------------+-------------------------+-----------------------|
| Erumperator |      1561 |           1.37 |        2134.86 |          2023.773 |         202.38 | 6 days, 3:43:00 |   431     0  1130  27.6 | 1752.507 USDT  40.89% |


In [42]:
TIMEFRAME="12h"
TIMERANGE=YTD

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@YTD without filters and protections{Style.RESET_ALL}")
print(update_params(23))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing hyperopted Erumperator on 12h@YTD without filters and protections
Params: BL 23
Strategy: buy on resistance_breakout and sell on breakout end
Function took 19.83 seconds to execute.
Backtested 2023-01-01 00:00:00 -> 2023-07-27 12:00:00 | Max open trades : 48
========================================================================== STRATEGY SUMMARY =========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |    Avg Duration |   Win  Draw  Loss  Win% |             Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+-----------------+-------------------------+----------------------|
| Erumperator |       227 |           1.99 |         451.49 |           441.823 |          44.18 | 6 days, 7:24:00 |    69     0   158  30.4 | 318.299 USDT  23.12% |


## 8h

In [35]:
TIMEFRAME="8h"
TIMERANGE=FULL

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@FULL without filters and protections{Style.RESET_ALL}")
print(update_params(27))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on support breakout{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing hyperopted Erumperator on 8h@FULL without filters and protections
Params: BL 27
Strategy: buy on resistance_breakout and sell on support breakout
Function took 18.78 seconds to execute.
Backtested 2021-08-08 00:00:00 -> 2022-01-06 00:00:00 | Max open trades : 37
========================================================================== STRATEGY SUMMARY =========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |     Avg Duration |   Win  Draw  Loss  Win% |            Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+------------------+-------------------------+---------------------|
| Erumperator |       146 |           7.68 |        1121.33 |          1039.793 |         103.98 | 7 days, 23:50:00 |    42     0   104  28.8 | 215.31 USDT  11.80% |


In [44]:
TIMEFRAME="8h"
TIMERANGE=R21TD

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@R21TD without filters and protections{Style.RESET_ALL}")
print(update_params(27))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on support breakout{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing hyperopted Erumperator on 8h@R21TD without filters and protections
Params: BL 27
Strategy: buy on resistance_breakout and sell on support breakout
Function took 1.30 minutes to execute.
Backtested 2021-01-15 16:00:00 -> 2023-07-27 16:00:00 | Max open trades : 48
=========================================================================== STRATEGY SUMMARY ==========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |     Avg Duration |   Win  Draw  Loss  Win% |              Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+------------------+-------------------------+-----------------------|
| Erumperator |      1468 |           2.63 |        3861.50 |          3846.873 |         384.69 | 8 days, 14:31:00 |   386     0  1082  26.3 | 1840.204 USDT  34.80% |


In [43]:
TIMEFRAME="8h"
TIMERANGE=YTD

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@YTD without filters and protections{Style.RESET_ALL}")
print(update_params(27))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on support breakout{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing hyperopted Erumperator on 8h@YTD without filters and protections
Params: BL 27
Strategy: buy on resistance_breakout and sell on support breakout
Function took 21.75 seconds to execute.
Backtested 2023-01-01 00:00:00 -> 2023-07-27 16:00:00 | Max open trades : 48
========================================================================== STRATEGY SUMMARY =========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |     Avg Duration |   Win  Draw  Loss  Win% |            Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+------------------+-------------------------+---------------------|
| Erumperator |       183 |           2.40 |         439.80 |           428.225 |          42.82 | 8 days, 10:50:00 |    58     0   125  31.7 | 220.94 USDT  16.68% |


# Add checks (particularly age_filter)

In [52]:
TIMEFRAME="1d"
TIMERANGE=R21TD

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@R21TD with checks and without protections{Style.RESET_ALL}")
print(update_params(17))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing hyperopted Erumperator on 1d@R21TD with checks and without protections
Params: BL 17
Strategy: buy on resistance_breakout and sell on breakout end
Function took 29.74 seconds to execute.
Backtested 2021-02-14 00:00:00 -> 2023-07-27 00:00:00 | Max open trades : 48
========================================================================== STRATEGY SUMMARY ==========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |     Avg Duration |   Win  Draw  Loss  Win% |             Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+------------------+-------------------------+----------------------|
| Erumperator |       684 |           2.52 |        1723.81 |          1701.763 |         170.18 | 10 days, 6:13:00 |   211     0   473  30.8 | 893.173 USDT  31.19% |


In [51]:
TIMEFRAME="12h"
TIMERANGE=R21TD

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@R21TD with checks and without protections{Style.RESET_ALL}")
print(update_params(23))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing hyperopted Erumperator on 12h@R21TD with checks and without protections
Params: BL 23
Strategy: buy on resistance_breakout and sell on breakout end
Function took 51.84 seconds to execute.
Backtested 2021-01-23 00:00:00 -> 2023-07-27 12:00:00 | Max open trades : 48
========================================================================== STRATEGY SUMMARY ==========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |    Avg Duration |   Win  Draw  Loss  Win% |              Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+-----------------+-------------------------+-----------------------|
| Erumperator |      1561 |           1.37 |        2134.86 |          2023.773 |         202.38 | 6 days, 3:43:00 |   431     0  1130  27.6 | 1752.507 USDT  40.89% |


In [53]:
TIMEFRAME="8h"
TIMERANGE=R21TD

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@R21TD with checks and without protections{Style.RESET_ALL}")
print(update_params(27))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on support breakout{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing hyperopted Erumperator on 8h@R21TD with checks and without protections
Params: BL 27
Strategy: buy on resistance_breakout and sell on support breakout
Function took 1.32 minutes to execute.
Backtested 2021-01-15 16:00:00 -> 2023-07-27 16:00:00 | Max open trades : 48
=========================================================================== STRATEGY SUMMARY ==========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |     Avg Duration |   Win  Draw  Loss  Win% |              Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+------------------+-------------------------+-----------------------|
| Erumperator |      1468 |           2.63 |        3861.50 |          3846.873 |         384.69 | 8 days, 14:31:00 |   386     0  1082  26.3 | 1840.204 USDT  34.80% |


# Hyperopt protections

In [ ]:
TIMEFRAME="1d"
TIMERANGE=FULL
SPACES="protection"
HYPEROPT_LOSS="ProfitSquareSortinoSpringEfficiencyHyperOptLoss"

print(f"Optimizing {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMERANGE}{Style.RESET_ALL}@FULL without filters or protections{Style.RESET_ALL}")
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade hyperopt --dmmp -j -2 -e 500 --disable-param-export --hyperopt-loss {HYPEROPT_LOSS} "\
    f"--strategy {STRATEGY} --config {MAINCONF} --spaces {SPACES} --timeframe {TIMEFRAME} --timerange {TIMERANGE}"

print(cmd)
output = run(cmd)
print(extract_best_result(output))

In [ ]:
TIMEFRAME="12h"
TIMERANGE=FULL
SPACES="protection"
HYPEROPT_LOSS="ProfitSquareSortinoSpringEfficiencyHyperOptLoss"

print(f"Optimizing {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMERANGE}{Style.RESET_ALL}@FULL without filters or protections{Style.RESET_ALL}")
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade hyperopt --dmmp -j -2 -e 500 --disable-param-export --hyperopt-loss {HYPEROPT_LOSS} "\
    f"--strategy {STRATEGY} --config {MAINCONF} --spaces {SPACES} --timeframe {TIMEFRAME} --timerange {TIMERANGE}"

print(cmd)
output = run(cmd)
print(extract_best_result(output))

In [ ]:
TIMEFRAME="8h"
TIMERANGE=FULL
SPACES="protection"
HYPEROPT_LOSS="ProfitSquareSortinoSpringEfficiencyHyperOptLoss"

print(f"Optimizing {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMERANGE}{Style.RESET_ALL}@FULL without filters or protections{Style.RESET_ALL}")
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade hyperopt --dmmp -j -2 -e 500 --disable-param-export --hyperopt-loss {HYPEROPT_LOSS} "\
    f"--strategy {STRATEGY} --config {MAINCONF} --spaces {SPACES} --timeframe {TIMEFRAME} --timerange {TIMERANGE}"

print(cmd)
output = run(cmd)
print(extract_best_result(output))

# Hyperopt trades

# Add protection

## 1d

In [54]:
TIMEFRAME="1d"
TIMERANGE=R21TD

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@R21TD with checks and without protections{Style.RESET_ALL}")
print(update_params(17))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing hyperopted Erumperator on 1d@R21TD with checks and without protections
Params: BL 17
Strategy: buy on resistance_breakout and sell on breakout end
Function took 48.12 seconds to execute.
Backtested 2021-02-14 00:00:00 -> 2023-07-27 00:00:00 | Max open trades : 48
========================================================================== STRATEGY SUMMARY ==========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |     Avg Duration |   Win  Draw  Loss  Win% |             Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+------------------+-------------------------+----------------------|
| Erumperator |       684 |           2.52 |        1723.81 |          1701.763 |         170.18 | 10 days, 6:13:00 |   211     0   473  30.8 | 893.173 USDT  31.19% |


In [55]:
TIMEFRAME="1d"
TIMERANGE=R21TD

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@R21TD with checks and with protections{Style.RESET_ALL}")
print(update_params(17))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month --enable-protections"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing hyperopted Erumperator on 1d@R21TD with checks and without protections
Params: BL 17
Strategy: buy on resistance_breakout and sell on breakout end
Function took 30.24 seconds to execute.
Backtested 2021-02-14 00:00:00 -> 2023-07-27 00:00:00 | Max open trades : 48
========================================================================== STRATEGY SUMMARY ==========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |     Avg Duration |   Win  Draw  Loss  Win% |             Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+------------------+-------------------------+----------------------|
| Erumperator |       344 |           6.94 |        2387.29 |          2385.374 |         238.54 | 12 days, 4:40:00 |   106     0   238  30.8 | 415.434 USDT  12.12% |


In [60]:
TIMEFRAME="1d"
TIMERANGE=YTD

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@YTD with checks and without protections{Style.RESET_ALL}")
print(update_params(17))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing hyperopted Erumperator on 1d@R21TD with checks and without protections
Params: BL 17
Strategy: buy on resistance_breakout and sell on breakout end
Function took 17.52 seconds to execute.
Backtested 2023-01-01 00:00:00 -> 2023-07-27 00:00:00 | Max open trades : 48
=========================================================================== STRATEGY SUMMARY ==========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |      Avg Duration |   Win  Draw  Loss  Win% |             Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+-------------------+-------------------------+----------------------|
| Erumperator |       138 |           2.75 |         380.15 |           372.082 |          37.21 | 10 days, 13:13:00 |    47     0    91  34.1 | 223.477 USDT  17.32% |


In [57]:
TIMEFRAME="1d"
TIMERANGE=YTD

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@YTD with checks and with protections{Style.RESET_ALL}")
print(update_params(17))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month --enable-protections"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing hyperopted Erumperator on 1d@R21TD with checks and without protections
Params: BL 17
Strategy: buy on resistance_breakout and sell on breakout end
Function took 17.48 seconds to execute.
Backtested 2023-01-01 00:00:00 -> 2023-07-27 00:00:00 | Max open trades : 48
=========================================================================== STRATEGY SUMMARY ==========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |      Avg Duration |   Win  Draw  Loss  Win% |             Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+-------------------+-------------------------+----------------------|
| Erumperator |        85 |           5.81 |         494.09 |           482.726 |          48.27 | 10 days, 15:49:00 |    31     0    54  36.5 | 207.306 USDT  14.93% |


## 12h

In [108]:
TIMEFRAME="12h"
TIMERANGE=R21TD

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@R21TD with checks and without protections{Style.RESET_ALL}")
print(update_params(23))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing hyperopted Erumperator on 12h@R21TD with checks and without protections
Params: BL 23
Strategy: buy on resistance_breakout and sell on breakout end
Function took 54.44 seconds to execute.
Backtested 2021-01-23 00:00:00 -> 2023-07-27 12:00:00 | Max open trades : 48
========================================================================== STRATEGY SUMMARY ==========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |    Avg Duration |   Win  Draw  Loss  Win% |              Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+-----------------+-------------------------+-----------------------|
| Erumperator |      1561 |           1.37 |        2134.86 |          2023.773 |         202.38 | 6 days, 3:43:00 |   431     0  1130  27.6 | 1752.507 USDT  40.89% |


In [109]:
TIMEFRAME="12h"
TIMERANGE=R21TD

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@R21TD with checks and with protections{Style.RESET_ALL}")
print(update_params(23))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month --enable-protections"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing hyperopted Erumperator on 12h@R21TD with checks and with protections
Params: BL 23
Strategy: buy on resistance_breakout and sell on breakout end
Function took 29.68 seconds to execute.
Backtested 2021-01-23 00:00:00 -> 2023-07-27 12:00:00 | Max open trades : 48
========================================================================== STRATEGY SUMMARY ==========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |     Avg Duration |   Win  Draw  Loss  Win% |             Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+------------------+-------------------------+----------------------|
| Erumperator |       402 |          -0.19 |         -75.48 |           -57.124 |          -5.71 | 6 days, 21:17:00 |   112     0   290  27.9 | 900.433 USDT  59.78% |


In [106]:
TIMEFRAME="12h"
TIMERANGE=YTD

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@YTD with checks and without protections{Style.RESET_ALL}")
print(update_params(23))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing hyperopted Erumperator on 12h@YTD with checks and without protections
Params: BL 23
Strategy: buy on resistance_breakout and sell on breakout end
Function took 20.04 seconds to execute.
Backtested 2023-01-01 00:00:00 -> 2023-07-27 12:00:00 | Max open trades : 48
========================================================================== STRATEGY SUMMARY =========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |    Avg Duration |   Win  Draw  Loss  Win% |             Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+-----------------+-------------------------+----------------------|
| Erumperator |       227 |           1.99 |         451.49 |           441.823 |          44.18 | 6 days, 7:24:00 |    69     0   158  30.4 | 318.299 USDT  23.12% |


In [107]:
TIMEFRAME="12h"
TIMERANGE=YTD

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@YTD with checks and with protections{Style.RESET_ALL}")
print(update_params(23))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month --enable-protections"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing hyperopted Erumperator on 12h@YTD with checks and with protections
Params: BL 23
Strategy: buy on resistance_breakout and sell on breakout end
Function took 18.78 seconds to execute.
Backtested 2023-01-01 00:00:00 -> 2023-07-27 12:00:00 | Max open trades : 48
========================================================================== STRATEGY SUMMARY ==========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |     Avg Duration |   Win  Draw  Loss  Win% |             Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+------------------+-------------------------+----------------------|
| Erumperator |       132 |           2.21 |         291.46 |           283.439 |          28.34 | 5 days, 23:33:00 |    39     0    93  29.5 | 242.068 USDT  17.63% |


## 8h

In [62]:
TIMEFRAME="8h"
TIMERANGE=R21TD

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@R21TD with checks and without protections{Style.RESET_ALL}")
print(update_params(17))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on start_support_breakout{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing hyperopted Erumperator on 8h@R21TD with checks and without protections
Params: BL 17
Strategy: buy on resistance_breakout and sell on start_support_breakout
Function took 1.30 minutes to execute.
Backtested 2021-01-15 16:00:00 -> 2023-07-27 16:00:00 | Max open trades : 48
========================================================================== STRATEGY SUMMARY ==========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |    Avg Duration |   Win  Draw  Loss  Win% |              Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+-----------------+-------------------------+-----------------------|
| Erumperator |      1630 |           1.56 |        2537.93 |          2481.953 |         248.20 | 7 days, 2:18:00 |   455     0  1175  27.9 | 1113.809 USDT  27.41% |


In [63]:
TIMEFRAME="8h"
TIMERANGE=R21TD

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@R21TD with checks and with protections{Style.RESET_ALL}")
print(update_params(17))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on start_support_breakout{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month --enable-protections"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing hyperopted Erumperator on 8h@R21TD with checks and with protections
Params: BL 17
Strategy: buy on resistance_breakout and sell on start_support_breakout
Function took 42.64 seconds to execute.
Backtested 2021-01-15 16:00:00 -> 2023-07-27 16:00:00 | Max open trades : 48
========================================================================= STRATEGY SUMMARY =========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |     Avg Duration |   Win  Draw  Loss  Win% |           Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+------------------+-------------------------+--------------------|
| Erumperator |       503 |           1.12 |         565.09 |           521.003 |          52.10 | 7 days, 12:22:00 |   140     0   363  27.8 | 573.5 USDT  36.83% |


In [64]:
TIMEFRAME="8h"
TIMERANGE=YTD

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@YTD with checks and without protections{Style.RESET_ALL}")
print(update_params(17))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on start_support_breakout{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing hyperopted Erumperator on 8h@YTD with checks and without protections
Params: BL 17
Strategy: buy on resistance_breakout and sell on start_support_breakout
Function took 23.33 seconds to execute.
Backtested 2023-01-01 00:00:00 -> 2023-07-27 16:00:00 | Max open trades : 48
========================================================================== STRATEGY SUMMARY ==========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |     Avg Duration |   Win  Draw  Loss  Win% |             Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+------------------+-------------------------+----------------------|
| Erumperator |       232 |           2.57 |         595.78 |           585.675 |          58.57 | 7 days, 13:14:00 |    78     0   154  33.6 | 271.947 USDT  17.37% |


In [65]:
TIMEFRAME="8h"
TIMERANGE=YTD

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@YTD with checks and with protections{Style.RESET_ALL}")
print(update_params(17))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on start_support_breakout{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --breakdown month --enable-protections"

output = run(cmd)
print(extract_summary(output))
# print(extract_monthly(output))

Testing hyperopted Erumperator on 8h@YTD with checks and with protections
Params: BL 17
Strategy: buy on resistance_breakout and sell on start_support_breakout
Function took 21.48 seconds to execute.
Backtested 2023-01-01 00:00:00 -> 2023-07-27 16:00:00 | Max open trades : 48
========================================================================== STRATEGY SUMMARY ==========================================================================
|    Strategy |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |     Avg Duration |   Win  Draw  Loss  Win% |             Drawdown |
|-------------+-----------+----------------+----------------+-------------------+----------------+------------------+-------------------------+----------------------|
| Erumperator |       114 |           4.59 |         523.04 |           514.882 |          51.49 | 9 days, 17:28:00 |    35     0    79  30.7 | 264.847 USDT  16.90% |


# Find blacklisting pairs

In [98]:
TIMEFRAME="1d"
TIMERANGE=FULL

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@FULL with checks and with protections{Style.RESET_ALL}")
print(update_params(17))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --enable-protections"

output = run(cmd)
print(extract_worst_pairs(output))

Testing hyperopted Erumperator on 1d@FULL with checks and with protections
Params: BL 17
Strategy: buy on resistance_breakout and sell on breakout end
Function took 11.32 seconds to execute.

================================================================== BACKTESTING REPORT =================================================================
|               Pair |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |      Avg Duration |   Win  Draw  Loss  Win% |
|--------------------+-----------+----------------+----------------+-------------------+----------------+-------------------+-------------------------|
|     AAVE/USDT:USDT |         3 |          -6.35 |         -19.05 |           -16.187 |          -1.62 |   1 day, 16:00:00 |     0     0     3     0 |
|     LINK/USDT:USDT |         3 |          -5.93 |         -17.79 |           -17.757 |          -1.78 |  10 days, 8:00:00 |     1     0     2  33.3 |
|      ETC/USDT:USDT |         3 |          -6.8

In [99]:
TIMEFRAME="1d"
TIMERANGE=R21TD

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@R21TD with checks and with protections{Style.RESET_ALL}")
print(update_params(17))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --enable-protections"

output = run(cmd)
print(extract_worst_pairs(output))

Testing hyperopted Erumperator on 1d@R21TD with checks and with protections
Params: BL 17
Strategy: buy on resistance_breakout and sell on breakout end
Function took 26.45 seconds to execute.

================================================================== BACKTESTING REPORT =================================================================
|               Pair |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |      Avg Duration |   Win  Draw  Loss  Win% |
|--------------------+-----------+----------------+----------------+-------------------+----------------+-------------------+-------------------------|
|      ADA/USDT:USDT |         8 |          -8.42 |         -67.35 |           -66.889 |          -6.69 |  7 days, 12:00:00 |     2     0     6  25.0 |
|     AVAX/USDT:USDT |        10 |          -8.53 |         -85.25 |           -67.043 |          -6.70 |  5 days, 16:48:00 |     3     0     7  30.0 |
|     AAVE/USDT:USDT |        12 |          -6.

In [100]:
TIMEFRAME="1d"
TIMERANGE=YTD

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@YTD with checks and with protections{Style.RESET_ALL}")
print(update_params(17))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --enable-protections"

output = run(cmd)
print(extract_worst_pairs(output))

Testing hyperopted Erumperator on 1d@YTD with checks and with protections
Params: BL 17
Strategy: buy on resistance_breakout and sell on breakout end
Function took 16.58 seconds to execute.

================================================================== BACKTESTING REPORT =================================================================
|               Pair |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |      Avg Duration |   Win  Draw  Loss  Win% |
|--------------------+-----------+----------------+----------------+-------------------+----------------+-------------------+-------------------------|
|      CRV/USDT:USDT |         2 |          -8.15 |         -16.30 |           -16.295 |          -1.63 |  6 days, 12:00:00 |     0     0     2     0 |
|      XRP/USDT:USDT |         2 |          -8.69 |         -17.38 |           -17.380 |          -1.74 | 21 days, 12:00:00 |     0     0     2     0 |
|     AAVE/USDT:USDT |         3 |          -6.38

In [134]:
TIMEFRAME="12h"
TIMERANGE=FULL

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@FULL with checks and with protections{Style.RESET_ALL}")
print(update_params(23))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --enable-protections"

output = run(cmd)
print(extract_worst_pairs(output))

Testing hyperopted Erumperator on 12h@FULL with checks and with protections
Params: BL 23
Strategy: buy on resistance_breakout and sell on breakout end
Function took 17.66 seconds to execute.

================================================================== BACKTESTING REPORT =================================================================
|               Pair |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |      Avg Duration |   Win  Draw  Loss  Win% |
|--------------------+-----------+----------------+----------------+-------------------+----------------+-------------------+-------------------------|
|     MASK/USDT:USDT |         7 |          -3.97 |         -27.77 |           -27.196 |          -2.72 |  3 days, 10:17:00 |     2     0     5  28.6 |
|     STMX/USDT:USDT |         4 |          -7.85 |         -31.41 |           -31.407 |          -3.14 |    1 day, 6:00:00 |     0     0     4     0 |
|     COMP/USDT:USDT |         7 |          -4.

In [135]:
TIMEFRAME="12h"
TIMERANGE=R21TD

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@R21TD with checks and with protections{Style.RESET_ALL}")
print(update_params(23))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --enable-protections"

output = run(cmd)
print(extract_worst_pairs(output))

Testing hyperopted Erumperator on 12h@R21TD with checks and with protections
Params: BL 23
Strategy: buy on resistance_breakout and sell on breakout end
Function took 52.11 seconds to execute.

================================================================== BACKTESTING REPORT =================================================================
|               Pair |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |      Avg Duration |   Win  Draw  Loss  Win% |
|--------------------+-----------+----------------+----------------+-------------------+----------------+-------------------+-------------------------|
|      CRV/USDT:USDT |        46 |          -2.25 |        -103.53 |          -103.355 |         -10.34 |  5 days, 10:57:00 |    12     0    34  26.1 |
|      EOS/USDT:USDT |        42 |          -2.80 |        -117.67 |          -117.488 |         -11.75 |  5 days, 15:26:00 |    11     0    31  26.2 |
|     BAKE/USDT:USDT |        31 |          -4

In [136]:
TIMEFRAME="12h"
TIMERANGE=YTD

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@YTD with checks and with protections{Style.RESET_ALL}")
print(update_params(23))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on breakout end{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --enable-protections"

output = run(cmd)
print(extract_worst_pairs(output))

Testing hyperopted Erumperator on 12h@YTD with checks and with protections
Params: BL 23
Strategy: buy on resistance_breakout and sell on breakout end
Function took 20.73 seconds to execute.

================================================================== BACKTESTING REPORT =================================================================
|               Pair |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |      Avg Duration |   Win  Draw  Loss  Win% |
|--------------------+-----------+----------------+----------------+-------------------+----------------+-------------------+-------------------------|
|      EOS/USDT:USDT |         3 |          -7.36 |         -22.09 |           -22.080 |          -2.21 |  5 days, 12:00:00 |     0     0     3     0 |
|      CRV/USDT:USDT |         7 |          -3.49 |         -24.45 |           -24.438 |          -2.44 |   3 days, 5:09:00 |     1     0     6  14.3 |
|      CFX/USDT:USDT |         4 |          -7.3

In [133]:
TIMEFRAME="8h"
TIMERANGE=FULL

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@FULL with checks and with protections{Style.RESET_ALL}")
print(update_params(27))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on start_support_breakout{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --enable-protections"

output = run(cmd)
print(extract_worst_pairs(output))

Testing hyperopted Erumperator on 8h@FULL with checks and with protections
Params: BL 27
Strategy: buy on resistance_breakout and sell on start_support_breakout
Function took 11.85 seconds to execute.

================================================================== BACKTESTING REPORT =================================================================
|               Pair |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |      Avg Duration |   Win  Draw  Loss  Win% |
|--------------------+-----------+----------------+----------------+-------------------+----------------+-------------------+-------------------------|
|     TOMO/USDT:USDT |         6 |          -4.61 |         -27.64 |           -27.106 |          -2.71 |   8 days, 4:00:00 |     2     0     4  33.3 |
|      OGN/USDT:USDT |         5 |          -6.67 |         -33.36 |           -33.168 |          -3.32 |    1 day, 4:48:00 |     1     0     4  20.0 |
|      CRV/USDT:USDT |        11 |    

In [132]:
TIMEFRAME="12h"
TIMERANGE=R21TD

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@R21TD with checks and with protections{Style.RESET_ALL}")
print(update_params(27))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on start_support_breakout{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --enable-protections"

output = run(cmd)
print(extract_worst_pairs(output))

Testing hyperopted Erumperator on 12h@R21TD with checks and with protections
Params: BL 27
Strategy: buy on resistance_breakout and sell on start_support_breakout
Function took 47.92 seconds to execute.

================================================================== BACKTESTING REPORT =================================================================
|               Pair |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |      Avg Duration |   Win  Draw  Loss  Win% |
|--------------------+-----------+----------------+----------------+-------------------+----------------+-------------------+-------------------------|
|     AVAX/USDT:USDT |        18 |          -6.53 |        -117.55 |          -101.468 |         -10.15 | 11 days, 14:40:00 |     5     0    13  27.8 |
|      CRV/USDT:USDT |        26 |          -4.10 |        -106.52 |          -106.377 |         -10.64 | 11 days, 13:23:00 |     7     0    19  26.9 |
|      DOT/USDT:USDT |        16 |  

In [131]:
TIMEFRAME="12h"
TIMERANGE=YTD

print(f"Testing hyperopted {Fore.BLUE}{STRATEGY}{Style.RESET_ALL} on {Fore.BLUE}{TIMEFRAME}@YTD with checks and with protections{Style.RESET_ALL}")
print(update_params(27))
print(f"Strategy: {Fore.GREEN}buy on resistance_breakout{Style.RESET_ALL} and {Fore.RED}sell on start_support_breakout{Style.RESET_ALL}")

cmd=f"freqtrade backtesting --dmmp --strategy {STRATEGY} --config {MAINCONF} --timeframe {TIMEFRAME} "\
    f"--timerange {TIMERANGE} --enable-protections"

output = run(cmd)
print(extract_worst_pairs(output))

Testing hyperopted Erumperator on 12h@YTD with checks and with protections
Params: BL 27
Strategy: buy on resistance_breakout and sell on start_support_breakout
Function took 20.28 seconds to execute.

================================================================== BACKTESTING REPORT =================================================================
|               Pair |   Entries |   Avg Profit % |   Cum Profit % |   Tot Profit USDT |   Tot Profit % |      Avg Duration |   Win  Draw  Loss  Win% |
|--------------------+-----------+----------------+----------------+-------------------+----------------+-------------------+-------------------------|
|     CELO/USDT:USDT |         2 |          -9.20 |         -18.40 |           -18.395 |          -1.84 |  2 days, 12:00:00 |     0     0     2     0 |
|      ARB/USDT:USDT |         2 |          -9.71 |         -19.42 |           -19.400 |          -1.94 |  16 days, 6:00:00 |     0     0     2     0 |
|      LTC/USDT:USDT |         6 |    

# Final test